In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
import astropy.units as u

plt.rcParams['figure.figsize'] = (20, 10)

In [2]:
def read_el(path):
    t = Table.read(path, format='ascii', names=('line', 'lambda', 'ratio'))
    t.columns[0].unit = u.AA
    return t

def read_sed(path):
    t = Table.read(path, format='ascii', names=('lambda', 'flux'))
    t.columns[0].unit = u.AA
    t.columns[1].unit = u.uJy
    return t

In [4]:
el_audrey = read_el('/home/aalvarez/Work/Projects/Phosphoros/Phosphoros/EmissionLines/auxdir/EmissionLines/O2_lines.txt')
el_stephane = read_el('/home/aalvarez/Work/Projects/Phosphoros/Phosphoros/EmissionLines/auxdir/EmissionLines/Ha_lines.txt')

In [16]:
sed_audrey = read_sed('SEDs_Audrey/CosmosSB_el/SB1_A_0.sed')
sed_step = read_sed('SEDs_Stephane/CosmosSB_el/SB1_A_0.sed')

In [17]:
def equivalent_width(el, sed):
    # https://en.wikipedia.org/wiki/Equivalent_width
    eps = 1e-5

    lines_dict = dict()
    for line_name, l, _ in el:
        # Find the line position
        idx = np.argmax(np.logical_and(sed['lambda'] >= l - eps, sed['lambda'] <= l + eps))
        # Get the reference value
        F0 = sed['flux'][idx - 1] * sed['flux'].unit
        # Compute W_\lambda
        Fl = sed['flux'][idx - 1 : idx + 1]
        dl = sed['lambda'][idx - 1 : idx + 1]
        Wl = np.trapz(1 - Fl/F0, dl)
        # Emission, make sure it is negative
        assert Wl < 0
        assert Wl.unit == u.AA
        Wl = np.abs(Wl)
        if line_name not in lines_dict:
            lines_dict[line_name] = []
        lines_dict[line_name].append(Wl)
    for k in lines_dict.keys():
        lines_dict[k] = u.Quantity(lines_dict[k])
    return lines_dict

In [18]:
def print_eq_dict(d):
    for k in sorted(d.keys()):
        print(f'{k:10}', end='\t')
        for v in d[k]:
            print(f'{v:.3f}', end='\t')
        print()

In [19]:
ew_audrey = equivalent_width(el_audrey, sed_audrey)
print_eq_dict(ew_audrey)

H_alpha   	13.893 Angstrom	
H_beta    	3.240 Angstrom	
H_delta   	0.751 Angstrom	
H_gamma   	1.343 Angstrom	
[OIII]    	0.840 Angstrom	2.539 Angstrom	
[OII]     	11.098 Angstrom	


In [20]:
ew_step = equivalent_width(el_stephane, sed_step)
print_eq_dict(ew_step)

H_alpha   	24.317 Angstrom	
H_beta    	5.653 Angstrom	
H_delta   	1.366 Angstrom	
H_gamma   	2.522 Angstrom	
[OIII]    	0.811 Angstrom	2.567 Angstrom	
[OII]     	8.304 Angstrom	8.330 Angstrom	


In [21]:
print('Ratios')
print('======')
for k in sorted(ew_audrey.keys()):
    aws = ew_audrey[k]
    sws = ew_step[k]
    if len(aws) == len(sws):
        for aw, sw in zip(aws, sws):
            print(f'{k:10}\t{aw/sw:.3f}')
    else:
        aw, sw = np.sum(aws), np.sum(sws)
        print(f'sum({k})\t{aw/sw:.4f}')

Ratios
H_alpha   	0.571
H_beta    	0.573
H_delta   	0.550
H_gamma   	0.533
[OIII]    	1.036
[OIII]    	0.989
sum([OII])	0.6672
